In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [16]:
# Specify the file path
file_path_test = "unlabelled_test_data.csv"
file_path_training_processed = "training_data_processed.csv"

# Read the CSV file
unlabelled_test_data = pd.read_csv(file_path_test)
training_data = pd.read_csv(file_path_training_processed)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(training_data['processed_sentence'])
y = training_data['difficulty']

In [18]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [19]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Assuming the training data is loaded into a DataFrame named 'training_data'
# with columns 'sentence' and 'difficulty'

# Label encoding the 'difficulty' column
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(training_data['difficulty'])

# Converting the encoded labels to categorical format
y_categorical = to_categorical(encoded_labels)

# Using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(training_data['processed_sentence']).toarray()

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.20, random_state=42)

# Building the Neural Network Model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')  # Output layer
])

# Compiling the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the Model
history = model.fit(X_train, y_train, epochs=6, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

/Users/oanaalexandrablanc/opt/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/oanaalexandrablanc/opt/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Epoch 1/6
240/240 [==============================] - 2s 5ms/step - loss: 1.5963 - accuracy: 0.3221 - val_loss: 1.3357 - val_accuracy: 0.4083
Epoch 2/6
240/240 [==============================] - 1s 3ms/step - loss: 1.0206 - accuracy: 0.6307 - val_loss: 1.2556 - val_accuracy: 0.4708
Epoch 3/6
240/240 [==============================] - 1s 3ms/step - loss: 0.5050 - accuracy: 0.8432 - val_loss: 1.4420 - val_accuracy: 0.4531
Epoch 4/6
240/240 [==============================] - 1s 3ms/step - loss: 0.2500 - accuracy: 0.9281 - val_loss: 1.6770 - val_accuracy: 0.4573
Epoch 5/6
240/240 [==============================] - 1s 3ms/step - loss: 0.1475 - accuracy: 0.9609 - val_loss: 1.8917 - val_accuracy: 0.4573
Epoch 6/6
30/30 [==============================] - 0s 2ms/step - loss: 2.0995 - accuracy: 0.4469
Accuracy: 0.4468750059604645


In [20]:
from sklearn.preprocessing import StandardScaler
import numpy as np

X_tfidf = vectorizer.fit_transform(training_data['processed_sentence']).toarray()

scaler = StandardScaler()
additional_features = scaler.fit_transform(training_data[['sentence_length', 'avg_word_length', 'type_token_ratio', 'syntactic_complexity', 'PUNCT', 'ADV', 'CCONJ', 'X', 'AUX', 'DET', 'PRON', 'NUM', 'NOUN', 'INTJ', 'ADP', 'ADJ', 'VERB', 'PROPN', 'SCONJ']])

# Standardize these features
additional_features_scaled = scaler.fit_transform(additional_features)

# Combine with TF-IDF features
X_combined = np.hstack([X_tfidf, additional_features_scaled])

/Users/oanaalexandrablanc/opt/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/oanaalexandrablanc/opt/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/oanaalexandrablanc/opt/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/oanaalexandrablanc/opt/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/utils/validation.py:767

In [21]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np

# Number of folds
num_folds = 5

# K-Fold Cross-Validation
kfold = KFold(n_splits=num_folds, shuffle=True)

# Assuming 'X_combined' is your feature matrix and 'encoded_labels' are your labels
accuracies = []

for train, test in kfold.split(X_combined, encoded_labels):
    # Split data into training and testing sets
    X_train, X_test = X_combined[train], X_combined[test]
    y_train, y_test = encoded_labels[train], encoded_labels[test]

    # Convert labels to categorical
    y_train_categorical = to_categorical(y_train)
    y_test_categorical = to_categorical(y_test)

    # Define the model
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(y_train_categorical.shape[1], activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train_categorical, epochs=6, batch_size=16)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test_categorical)
    accuracies.append(accuracy)

# Calculate the average and standard deviation of the accuracies
avg_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
print(f"Average accuracy: {avg_accuracy}, Standard Deviation: {std_accuracy}")

Epoch 1/6
240/240 [==============================] - 1s 2ms/step - loss: 1.4037 - accuracy: 0.3964
Epoch 2/6
240/240 [==============================] - 1s 2ms/step - loss: 1.0772 - accuracy: 0.5669
Epoch 3/6
240/240 [==============================] - 1s 2ms/step - loss: 0.7057 - accuracy: 0.7578
Epoch 4/6
240/240 [==============================] - 1s 3ms/step - loss: 0.3516 - accuracy: 0.8971
Epoch 5/6
240/240 [==============================] - 1s 4ms/step - loss: 0.1797 - accuracy: 0.9523
Epoch 6/6
30/30 [==============================] - 0s 1ms/step - loss: 1.6980 - accuracy: 0.4635
Epoch 1/6
240/240 [==============================] - 2s 3ms/step - loss: 1.4079 - accuracy: 0.3839
Epoch 2/6
240/240 [==============================] - 2s 7ms/step - loss: 1.0501 - accuracy: 0.5792
Epoch 3/6
240/240 [==============================] - 1s 3ms/step - loss: 0.6624 - accuracy: 0.7820
Epoch 4/6
240/240 [==============================] - 1s 2ms/step - loss: 0.3514 - accuracy: 0.8943
Epoch 5/6
24